# From tables to graphs (draft version)

In [1]:
import pandas as pd
import numpy as np
import networkx as nx

# 1. Loading the data

In [2]:
comments = pd.read_csv('clean_data/comments_ml.csv',parse_dates=[-3]).drop('Unnamed: 0',axis=1)
votes = pd.read_csv('clean_data/votes_ml.csv',parse_dates=[-4]).drop('Unnamed: 0',axis=1)
inters = pd.read_csv('clean_data/interactions_ml.csv').drop('Unnamed: 0',axis=1)
target = pd.read_csv('clean_data/target_ml.csv').rename(columns={'votes_num':'target'})

# 2. Employee Features

## 2.1 Vote features

In [3]:

vote_feats = votes.groupby(['uid'])['vote'].agg({'votes_1': lambda x: len(x[x==1]),
                                                        'votes_2': lambda x: len(x[x==2]),
                                                        'votes_3': lambda x: len(x[x==3]),
                                                        'votes_4': lambda x: len(x[x==4]),
                                                        'votes_mean': lambda x: x.mean(),
                                                        'votes_std': lambda x: x.std(),
                                                       'votes_num': lambda x: len(x),
                                                       }).copy().fillna(0)

## 2.2 Comments features

In [4]:
likes_feats = comments.groupby(['uid'])['likes'].agg({
                                          'likes_num': lambda x: len(x),
                                          'likes_mean': lambda x: x.mean(),
                                          'likes_std': lambda x: x.std(),
                                          'likes_sum': lambda x: x.sum(),
                                                       })
dislikes_feats = comments.groupby(['uid'])['dislikes'].agg({
                                          'dislikes_num': lambda x: len(x),
                                          'dislikes_mean': lambda x: x.mean(),
                                          'dislikes_std': lambda x: x.std(),
                                          'dislikes_sum': lambda x: x.sum(),
                                                       })
coms_feats = comments.dropna().groupby(['uid'])['comment'].agg({'com_num': lambda x: len(x),
                                                             'com_mean': lambda x: x.apply(len).mean(),
                                                             'com_std': lambda x: x.map(len).std(),
                                                             'com_sum': lambda x: x.map(len).sum()})

## 2.3 Merge all the features

In [5]:
features = pd.merge(vote_feats,likes_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,dislikes_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,coms_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,target,left_index=True,right_on='uid',how='right')#fill users with 0 comments
E_features = features.copy()
E_features.rename(columns=dict([(x,'E_'+str(x)) for x in E_features.columns]),inplace=True)

In [19]:
E_features.to_csv('features/e_feats.csv')

In [7]:
features.columns

Index(['votes_2', 'votes_std', 'votes_3', 'votes_num', 'votes_mean', 'votes_1',
       'votes_4', 'likes_num', 'likes_std', 'likes_mean', 'likes_sum',
       'dislikes_mean', 'dislikes_std', 'dislikes_num', 'dislikes_sum',
       'com_std', 'com_mean', 'com_num', 'com_sum', 'uid', 'target'],
      dtype='object')

# Graph employee features

In [8]:
import networkx as nx
def link_all(users_index,comments,inter):
    """Returns a graph containing relationships of like/dislike among employees"""
    g = nx.DiGraph()
    #One node for every employee storing its employee and company ids
    for ix,x in users_index.iterrows():
        if not g.has_node(x['uid']):
            c,u = x['uid'].split('_')
            g.add_node(x['uid'],company=c,employee=u)
    #adding links to a graph based on likes/dislikes
    com_ids = comments['commentId'].unique()
    users = comments['uid'].unique().tolist()
    print("linking {} comments from {} users".format(len(com_ids),len(users)))
    for com in com_ids:
        #who frite current comment
        writer = comments[comments['commentId']==com]['uid'].values[0]
        #info about com
        df = inter[inter['commentId']==com]
        #people who disliked the comment
        haters = df[df['liked']==0]['uid'].values.copy().tolist()
        #people who like the comment
        likers = df[df['liked']==1]['uid'].values.copy().tolist()
        for i,u in enumerate(likers):
            if not g.has_edge(u,writer):
                g.add_edge(u,writer,int_sum=1,interactions=1,liked=1,disliked=0)
            else:
                g.edge[u][writer]['interactions'] += 1
                g.edge[u][writer]['int_sum'] += 1
                g.edge[u][writer]['liked'] += 1

        for i,u in enumerate(haters):
            if not g.has_edge(u,writer):
                g.add_edge(u,writer,int_sum=-1,interactions=1,liked=0,disliked=1)
            else:
                g.edge[u][writer]['interactions'] += 1
                g.edge[u][writer]['disliked'] += 1
                g.edge[u][writer]['int_sum'] -= 1
    return g     

def add_info_to_graph(features,G):
    """Adds the information contained in a DataFrame to a networkx graph"""
    for n in G.nodes_iter():
        fs = features.ix[n].to_dict()
        G.add_node(n,**fs)
    return G

def add_rel_likes_metrics(g):
    """Adds infomation to edges about the relative number of likes a given employee gave to another."""
    for src in g.nodes_iter():
        t_rel = 0
        neigh = nx.neighbors(g,src)
        f = 1 if len(neigh)==0 else len(neigh)
        for dst in neigh:
            rel = g.edge[src][dst]['liked']/g.edge[src][dst]['interactions']
            g.edge[src][dst]['rel_agree'] = rel
            t_rel += rel
        g.node[src]['mean_agree'] = t_rel/f

def calculate_metrics(g,metrics,weight=None):
    """Returns a DataFrame containing the metrics of a given graph.
    It will calculate the metrics both in the directed and  undirected version opf the graph"""
    graph_df = pd.DataFrame()
    nw =  '' if weight is None else weight+'_'
    for met in metrics:
        graph_df['G_'+nw+met.__name__] = pd.Series(met(g))
        graph_df['G_'+nw+'w_'+met.__name__] = pd.Series(met(g,weight=weight))
        graph_df['G_'+nw+met.__name__+'_u'] = pd.Series(met(g.to_undirected()))
        graph_df['G_'+nw+'w_'+met.__name__+'_u'] = pd.Series(met(g.to_undirected(),weight=weight))
    #Normalize metrics from 0 to 1
    graph_df = (graph_df-graph_df.min(axis=0))/(graph_df.max(axis=0)-graph_df.min(axis=0))
    return graph_df

from sklearn.decomposition import NMF
def get_clusters(g, weight=None,n_components=2,companyAlias=''):
    """Return a DataFrame containing the NMF clustering information of a given graph"""
    nw =  '' if weight is None else weight+'_'
    #Calculate on directed version of the graph
    model = NMF(n_components=n_components)
    X = nx.adjacency_matrix(g,weight=weight).todense()
    if (X < 0).any():
        X = np.abs(X)#NMF only accepts non-negative values
    communities = model.fit_transform(X)
    cd = pd.DataFrame(index=list(g.nodes_iter()),
                      columns=['G_'+nw+'NMF'+str(i)+'_d' for i in range(1,n_components+1)],
                      data=communities) 
    #Undirected version
    Xd = nx.adjacency_matrix(g.to_undirected(),weight=weight).todense()
    if (Xd < 0).any():
        Xd = np.abs(Xd)
    communities = model.fit_transform(Xd)
    clusters = cd.combine_first(pd.DataFrame(index=cd.index,
                                             columns=['G_'+nw+'NMF'+str(i)+'_u' for i in range(1,n_components+1)]
                                             ,data=communities))
    return clusters 

def calculate_graph_features(G,features):
    add_rel_likes_metrics(G)
    ng = add_info_to_graph(features=features,G=G.copy())
    
    met_funcs = [nx.degree, nx.betweenness_centrality]
    graph_met_rel = calculate_metrics(ng,met_funcs,weight='rel_agree')
    cluster_rel = get_clusters(ng,weight='rel_agree')
    agree_feats = pd.merge(graph_met_rel,cluster_rel,left_index=True,right_index=True,how='outer')
    graph_met_int = calculate_metrics(ng,met_funcs,weight='interactions')
    cluster_int = get_clusters(ng,weight='interactions')
    int_feats = pd.merge(graph_met_int,cluster_int,left_index=True,right_index=True,how='outer')
    graph_feats = pd.merge(int_feats,agree_feats,left_index=True,right_index=True,how='outer')
    #final_features = pd.merge(features,graph_feats,left_index=True,right_index=True,how='outer')
    ng = add_info_to_graph(features=graph_feats,G=ng.copy())
    
    return graph_feats,ng

In [9]:
G_raw = link_all(target,comments,inters)

linking 17919 comments from 900 users


In [16]:
EG_features,g_total = calculate_graph_features(G_raw.copy(),features.set_index('uid'))
EG_features.head()

,G_interactions_degree,G_interactions_w_degree,G_interactions_degree_u,G_interactions_w_degree_u,G_interactions_betweenness_centrality,G_interactions_w_betweenness_centrality,G_interactions_betweenness_centrality_u,G_interactions_w_betweenness_centrality_u,G_interactions_NMF1_d,G_interactions_NMF1_u,...,G_rel_agree_degree_u,G_rel_agree_w_degree_u,G_rel_agree_betweenness_centrality,G_rel_agree_w_betweenness_centrality,G_rel_agree_betweenness_centrality_u,G_rel_agree_w_betweenness_centrality_u,G_rel_agree_NMF1_d,G_rel_agree_NMF1_u,G_rel_agree_NMF2_d,G_rel_agree_NMF2_u
0_25,0.080952,0.052985,0.084615,0.066792,0.005056,0.010556,0.007113,0.006652,0.0,0.0,...,0.084615,0.098374,0.005056,1.389244e-07,0.007113,1.972912e-19,0.0,0.0,0.0,0.000000
0_259,0.090476,0.056870,0.084615,0.063670,0.008131,0.031115,0.007113,0.001765,0.0,0.0,...,0.084615,0.085043,0.008131,3.989625e-07,0.007113,7.842933e-19,0.0,0.0,0.0,0.774894
0_271,0.023810,0.004945,0.023077,0.003745,0.000000,0.002262,0.000000,0.000000,0.0,0.0,...,0.023077,0.021584,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000
0_277,0.061905,0.012363,0.061538,0.016854,0.000742,0.028250,0.000343,0.010724,0.0,0.0,...,0.061538,0.074004,0.000742,7.124331e-09,0.000343,0.000000e+00,0.0,0.0,0.0,0.000000
0_278,0.066667,0.024020,0.069231,0.026841,0.000816,0.002061,0.001114,0.001222,0.0,0.0,...,0.069231,0.080171,0.000816,0.000000e+00,0.001114,0.000000e+00,0.0,0.0,0.0,0.000000


In [17]:
EG_features.to_csv('features/eg_feats.csv')

# 3. Company features

## 3.1 Comapany vote features

In [18]:
comp_vote_feats = votes.groupby(['comid'])['vote'].agg({'votes_1': lambda x: len(x[x==1]),
                                                        'votes_2': lambda x: len(x[x==2]),
                                                        'votes_3': lambda x: len(x[x==3]),
                                                        'votes_4': lambda x: len(x[x==4]),
                                                        'votes_mean': lambda x: x.mean(),
                                                        'votes_std': lambda x: x.std(),
                                                       'votes_num': lambda x: len(x),
                                                       }).copy()

## 3.2 Comments features

In [12]:
comp_likes_feats = comments.groupby(['comid'])['likes'].agg({
                                          'likes_num': lambda x: len(x),
                                          'likes_mean': lambda x: x.mean(),
                                          'likes_std': lambda x: x.std(),
                                          'likes_sum': lambda x: x.sum(),
                                                       })
comp_dislikes_feats = comments.groupby(['comid'])['dislikes'].agg({
                                          'dislikes_num': lambda x: len(x),
                                          'dislikes_mean': lambda x: x.mean(),
                                          'dislikes_std': lambda x: x.std(),
                                          'dislikes_sum': lambda x: x.sum(),
                                                       })
comp_coms_feats = comments.dropna().groupby(['comid'])['comment'].agg({'com_num': lambda x: len(x),
                                                             'com_mean': lambda x: x.apply(len).mean(),
                                                             'com_std': lambda x: x.map(len).std(),
                                                             'com_sum': lambda x: x.map(len).sum()})

## 3.3 Merging and aggregating company features

In [22]:
comp_features = pd.merge(comp_vote_feats,comp_likes_feats,left_index=True,right_index=True,how='outer')
comp_features = pd.merge(comp_features,comp_dislikes_feats,left_index=True,right_index=True,how='outer')
comp_features = pd.merge(comp_features,comp_coms_feats,left_index=True,right_index=True,how='outer')
#features = pd.merge(features,users_index.set_index('uid')[['churn']],left_index=True,right_index=True,how='left')
#features.set_index('uid'),'ui
comp_df = comp_features.fillna(1)
C_features_raw = comp_df.applymap(lambda x: 1 if x==0 else x)


## 3.4 Creating company wide features

In [24]:
C_features = pd.DataFrame(index=target.uid,columns=C_features_raw.columns)
for i in range(len(target.uid)):
    company = int(C_features.index.values[i].split('_')[0])
    C_features.iloc[i,:] = C_features_raw.ix[company].values

In [25]:
C_features.to_csv('features/c_feats.csv')

# 4. Employee-Company features

In [47]:
features['comid'] = features['uid'].map(lambda x: x.split('_')[0] )
comments['comid'] = comments['uid'].map(lambda x: x.split('_')[0] )
inters['comid'] = inters['uid'].map(lambda x: x.split('_')[0] )
target['comid'] = target['uid'].map(lambda x: x.split('_')[0] )

In [48]:
rel_feats = features.set_index('uid')[C_features_raw.columns.values.tolist()+['comid']].copy()
companies = C_features_raw.index.values.tolist()
for comp in companies:
    rel_feats.loc[rel_feats['comid']==int(comp)] = rel_feats.loc[rel_feats['comid']==int(comp)]/comp_df.ix[comp]
#rel_feats.drop('comid',axis=1,inplace=True)

rel_feats.rename(columns=dict([(x,'CE_'+str(x)) for x in rel_feats.columns]),inplace=True)

CE_features = rel_feats.copy().fillna(0.)
CE_features.to_csv('features/ce_feats.csv')

# 5. Employee-Company Graph features

In [50]:
def one_company_graph_data(comp,target,comments,inters,features):
    g_comps = {}
    feats = pd.DataFrame()
    comp = str(comp)
    ui = target[target['comid']==comp].copy()
    co = comments[comments['comid']==comp].copy()
    ints = inters[inters['comid']==comp].copy()
    g_c = link_all(ui,co,ints)
    if g_c.node!={}:
        g_comps[comp] = g_c
        _features,g_c = calculate_graph_features(g_c,features[features['CE_comid'].astype(int)==int(comp)].copy())
        feats = _features.copy()
    return feats,g_comps

rgdf,g_comps  = one_company_graph_data(companies[0],target,comments,inters,rel_feats)

for comp in companies[1:]:
    print("calculating comp : {}".format(comp))
    _rgdf,_g_comps  = one_company_graph_data(comp,target,comments,inters,rel_feats)
    g_comps.update(_g_comps)
    rgdf = pd.concat([rgdf,_rgdf])

linking 213 comments from 10 users
calculating comp : 4
linking 2762 comments from 87 users
calculating comp : 6
linking 1760 comments from 78 users
calculating comp : 11
linking 3062 comments from 130 users
calculating comp : 12
linking 905 comments from 27 users
calculating comp : 13
linking 474 comments from 14 users
calculating comp : 16
linking 1691 comments from 99 users
calculating comp : 17
linking 789 comments from 39 users
calculating comp : 18
linking 1224 comments from 86 users
calculating comp : 19
linking 233 comments from 7 users
calculating comp : 20
linking 608 comments from 36 users
calculating comp : 21
linking 765 comments from 56 users
calculating comp : 22
linking 487 comments from 23 users
calculating comp : 23
linking 488 comments from 23 users
calculating comp : 24
linking 339 comments from 24 users
calculating comp : 25
linking 18 comments from 1 users
calculating comp : 26
linking 712 comments from 67 users
calculating comp : 29
linking 150 comments from 23 u

In [51]:
CEG_features = rgdf.rename(columns=dict([(x,'CE'+str(x)) for x in rgdf.columns])).copy()
CEG_features.to_csv('features/ceg_feats.csv')